# This is the LSTM quantile regressor

I think it would make sense to test agains some other quantile regressor, and I would quite like quantile-forest, or possibly catboost. The former is likely preferable due to its ease of training. The isotonic regression step should be superflous, and we could get as many quantiles as we like.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.isotonic import IsotonicRegression
from scipy.optimize import minimize_scalar
from tqdm.notebook import tqdm
from scipy.interpolate import interp1d
MACHINE_EPSILON = lambda x: np.abs(x) * np.finfo(np.float64).eps
from online_cp import PluginMartingale

def utility(flex, limit, demand, flex_cost=0.1, overdraw_cost=1):
    '''
    Simple utility function that takes into account the possibility to buy too little flex.
    NOTE: This is unrealistic in that we can not buy a real number of flex in practice.
    '''
    effective_demand = demand - flex # Are we above despide having flex?
    effective_above = effective_demand > limit 
    above = demand > limit # Would we have been above if we did not buy flex?
    if effective_above: # Even with flex, we are above the limit
        u = - overdraw_cost * (effective_demand - limit) - flex_cost * flex
    elif above: # We would have been over the limit if we had not had flex.
        u = overdraw_cost * (demand - limit) - flex_cost * flex
    else: # We are below no matter what.
        u = - flex_cost * flex
    return float(u)

# Load data

In [ ]:
lstm_data = 'lstm_quantile_regression.txt'

df_qr = pd.read_csv(lstm_data, parse_dates=True)
df_qr.index = pd.to_datetime(df_qr['timestamp_forecast'])
df_qr.timestamp_start = pd.to_datetime(df_qr.timestamp_start)
df_qr.timestamp_forecast = pd.to_datetime(df_qr.timestamp_forecast)

In [ ]:
df_qr.true_target.min(), df_qr.true_target.max()

In [ ]:
df_qr['p0.01'].min(), df_qr['p0.99'].max()

Based on the above, I think 10 and 60 are reasonable bounds

# Apply isontonic regression

In [ ]:
limit = 40
df_quantiles = df_qr.drop(columns=['station', 'timestamp_start', 'timestamp_forecast', 'true_target'])
df_quantiles.columns = sorted(df_quantiles.columns, key=lambda x: float(x[1:]))
sorted_columns = sorted(df_quantiles.columns, key=lambda x: float(x[1:]))
df_quantiles.columns = [float(col[1:]) for col in sorted_columns]

# Apply isontonic regression to get proper quantiles
ir = IsotonicRegression()
for i, row in df_quantiles.iterrows():
    ir.fit(df_quantiles.columns, row.values)
    row[row.index] = ir.predict(df_quantiles.columns)

# Write decision process for protected and basic decision process

In [ ]:
def exp_util(cpd, d, jump_points):
    Delta_Q = np.array([utility(d, limit, y) * (cpd(y=y + MACHINE_EPSILON(y)) - cpd(y=y - MACHINE_EPSILON(y))) for y in jump_points])
    return jump_points @ Delta_Q


def expectation(cpd, jump_points):
    Delta_Q = np.array([(cpd(y=y + MACHINE_EPSILON(y)) - cpd(y=y - MACHINE_EPSILON(y))) for y in jump_points])
    return jump_points @ Delta_Q

def make_decision(idx, B_n, lower_bound=0):
    # Define the quantile values
    quantiles = np.zeros(101)
    quantiles[1:-1] = df_quantiles.iloc[idx]
    quantiles[0] = 10 # Demand is never negative. We could probably take some safe lower bound here. NOTE: A tighter bound would be better.
    quantiles[-1] = 55 # Make it bounded. We could take some safe upper bound here. 2*limit is arbitrary, but probably safe. NOTE: A tighter bound would be better.

    # Define the probability levels corresponding to the quantiles
    p = np.zeros(101)  # Evenly spaced probabilities
    p[1:-1] = df_quantiles.columns
    p[0] = 0.0
    p[-1] = 1.0

    # Sort unique quantiles and associated probabilities
    unique_x, unique_p = np.unique(quantiles, return_index=True)
    sorted_p = p[unique_p]

    # Define exact generalized inverse (right-continuous)
    def exact_cdf(x):
        return sorted_p[np.searchsorted(unique_x, x, side='right') - 1]
    
    # Make base decision
    cdf = lambda y: exact_cdf(y)

    optimiser_decision = max(0, minimize_scalar(lambda d: -exp_util(cdf, d, unique_x), bounds=(lower_bound,10)).x)

    # Make protected decision
    protected_cdf = lambda y: B_n(exact_cdf(y))

    optimiser_protected_decision = max(0, minimize_scalar(lambda d: -exp_util(protected_cdf, d, unique_x), bounds=(lower_bound,10)).x)

    optimal_d = max(df_qr.iloc[idx].true_target - limit, 0)

    return optimal_d, optimiser_decision, optimiser_protected_decision, cdf, protected_cdf


In [ ]:
decisions = np.zeros(df_quantiles.shape[0])
optimal_decisions = np.zeros_like(decisions)
protected_decisions = np.zeros_like(decisions)

pit = np.zeros_like(decisions)
protected_pit = np.zeros_like(decisions)

martingale = PluginMartingale(warnings=False)

num = 700 # df_quantiles.shape[0]

for idx in tqdm(range(num)):
    optimal_d, optimiser_decision, optimiser_protected_decision, cdf, protected_cdf = make_decision(idx, martingale.B_n, lower_bound=-1)
    decisions[idx] = optimiser_decision
    optimal_decisions[idx] = optimal_d
    protected_decisions[idx] = optimiser_protected_decision
    y = df_qr.iloc[idx].true_target
    p = cdf(y)
    protected_p = martingale.B_n(p)
    pit[idx] = p
    protected_pit[idx] = protected_p

    martingale.update_martingale_value(p)

In [ ]:
epsilon = 0.05
base_decisions = (df_qr.drop(columns=['station', 'timestamp_start', 'timestamp_forecast', 'true_target'])[f'p{1-epsilon}'] - limit).clip(lower=0).values
median_decisions = (df_qr.drop(columns=['station', 'timestamp_start', 'timestamp_forecast', 'true_target'])['p0.5'] - limit).clip(lower=0).values
isotonic_decisions = (df_quantiles[1-epsilon] - limit).clip(lower=0).values

flex_cost = 0.025 # This is the realistic one, that we were using in the paper.
util_func = lambda d, demand: utility(d, limit, demand, flex_cost=flex_cost)

Utility = np.zeros_like(decisions)
for i, (d, demand) in enumerate(zip(decisions, df_qr.true_target.values)):
    Utility[i] = util_func(d, demand)

optimal_Utility = np.zeros_like(decisions)
for i, (d, demand) in enumerate(zip(optimal_decisions, df_qr.true_target.values)):
    optimal_Utility[i] = util_func(d, demand)

protected_Utility = np.zeros_like(decisions)
for i, (d, demand) in enumerate(zip(protected_decisions, df_qr.true_target.values)):
    protected_Utility[i] = util_func(d, demand)

base_Utility = np.zeros_like(decisions)
for i, (d, demand) in enumerate(zip(base_decisions, df_qr.true_target.values)):
    base_Utility[i] = util_func(d, demand)

isotonic_Utility = np.zeros_like(decisions)
for i, (d, demand) in enumerate(zip(isotonic_decisions, df_qr.true_target.values)):
    isotonic_Utility[i] = util_func(d, demand)

median_Utility = np.zeros_like(decisions)
for i, (d, demand) in enumerate(zip(median_decisions, df_qr.true_target.values)):
    median_Utility[i] = util_func(d, demand)

In [ ]:
Regret = optimal_Utility - Utility
protected_Regret = optimal_Utility - protected_Utility
base_Regret = optimal_Utility - base_Utility
isotonic_Regret = optimal_Utility - isotonic_Utility
median_Regret = optimal_Utility - median_Utility

plt.plot(Regret.cumsum(), label='Expectation')
plt.plot(protected_Regret.cumsum(), label=f'Protected expectation')
plt.plot(base_Regret.cumsum(), label=f'Base confidence {1-epsilon}')
plt.plot(isotonic_Regret.cumsum(), label=f'Isotonic confidence {1-epsilon}')
# plt.plot(median_Regret.cumsum(), label='Median')
plt.title('Cumulative regret')
plt.legend()

In [ ]:
print('Total utility:')

print(f'Optimal: {optimal_Utility.sum()}')
print(f'Expectation: {Utility.sum()}')
print(f'Protected expectation: {protected_Utility.sum()}')
print(f'Base confidence {1-epsilon}: {base_Utility.sum()}')
print(f'Isotonic confidence {1-epsilon}: {isotonic_Utility.sum()}')
print(f'Median: {median_Utility.sum()}')

In [ ]:
print('Number of flex instances:')

print(f'Optimal: {(optimal_decisions > 0).sum()}')
print(f'Expectation: {(decisions > 0).sum()}')
print(f'Protected expectation: {(protected_decisions > 0).sum()}')
print(f'Base confidence {1-epsilon}: {(base_decisions > 0).sum()}')
print(f'Isotonic confidence {1-epsilon}: {(isotonic_decisions > 0).sum()}')
print(f'Median: {(median_decisions > 0).sum()}')

In [ ]:
print('Number of overdraws:')

print(f'Optimal: {(df_qr.true_target.values - optimal_decisions > limit).sum()}')
print(f'Expectation: {(df_qr.true_target.values - decisions > limit).sum()}')
print(f'Protected expectation: {(df_qr.true_target.values - protected_decisions > limit).sum()}')
print(f'Base confidence {1-epsilon}: {(df_qr.true_target.values - base_decisions > limit).sum()}')
print(f'Isotonic confidence {1-epsilon}: {(df_qr.true_target.values - isotonic_decisions > limit).sum()}')
print(f'Median: {(df_qr.true_target.values - median_decisions > limit).sum()}')

In [ ]:
print('Number of unnecessary flex buys:')

print(f'Optimal: {((df_qr.true_target < limit) & (optimal_decisions > 0)).sum()}')
print(f'Expectation: {((df_qr.true_target < limit) & (decisions > 0)).sum()}')
print(f'Protected expectation: {((df_qr.true_target < limit) & (protected_decisions > 0)).sum()}')
print(f'Base confidence {1-epsilon}: {((df_qr.true_target < limit) & (base_decisions > 0)).sum()}')
print(f'Isotonic confidence {1-epsilon}: {((df_qr.true_target < limit) & (isotonic_decisions > 0)).sum()}')
print(f'Median: {((df_qr.true_target < limit) & (median_decisions > 0)).sum()}')

In [ ]:
print('Total flex:')

print(f'Optimal: {optimal_decisions.sum()}')
print(f'Expectation: {decisions.sum()}')
print(f'Protected expectation: {protected_decisions.sum()}')
print(f'Base confidence {1-epsilon}: {base_decisions.sum()}')
print(f'Isotonic confidence {1-epsilon}: {isotonic_decisions.sum()}')
print(f'Median: {median_decisions.sum()}')

In [ ]:
print('Total flex cost:')

print(f'Optimal: {optimal_decisions.sum() * flex_cost}')
print(f'Expectation: {decisions.sum() * flex_cost}')
print(f'Protected expectation: {protected_decisions.sum() * flex_cost}')
print(f'Base confidence {1-epsilon}: {base_decisions.sum() * flex_cost}')
print(f'Isotonic confidence {1-epsilon}: {isotonic_decisions.sum() * flex_cost}')
print(f'Median: {median_decisions.sum() * flex_cost}')

In [ ]:
print('Total overdraw cost:')
# TODO: Figure out

In [ ]:
plt.hist(pit, density=True)
plt.hist(protected_pit, density=True, alpha=0.5)